<a href="https://colab.research.google.com/github/sapkotashishir/Crop_map_2024/blob/main/Summer_Crop_type_mapping_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
import geemap

ee.Authenticate()
ee.Initialize(project='glodal-projects')
Map = geemap.Map()

In [ ]:
js_snippet = """

 // Create an export classifier task to run.
var assetId = 'users/ssapkota913/winter_crop_classifier' ;  // <> modify these
Export.classifier.toAsset({
   classifier: trainedClassifier,
   description: 'winter_crop_classifier',
   assetId: assetId
 });
"""

lines = geemap.js_snippet_to_py(js_snippet, add_new_cell=False, import_ee=True, import_geemap=True, show_map=True)
for line in lines:
    print(line.rstrip())

NameError: name 'geemap' is not defined

### Install geemap

In [ ]:
# !pip install geemap

### Import and authenticate gee

### Data preparation

*   Single Image creation for sampling the spectral and other features
*   Sampling points creation for model development



Single Image Creation from the assets that we have exported



In [ ]:
# Get the list of asset metadata
assetList = ee.data.listAssets("users/ssapkota913/kailali_summer")['assets']

# Extract only the 'id' field from each asset
image_ids = [asset['id'] for asset in assetList]

# Create an ImageCollection from the list of image IDs
image_collection = ee.ImageCollection(image_ids)

# Print the information of the ImageCollection
print(image_collection.getInfo())

{'type': 'ImageCollection', 'bands': [], 'features': [{'type': 'Image', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [9129, 7501], 'crs': 'EPSG:4326', 'crs_transform': [8.983152841195215e-05, 0, 80.46973567850056, 0, -8.983152841195215e-05, 29.068135121181538]}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [9129, 7501], 'crs': 'EPSG:4326', 'crs_transform': [8.983152841195215e-05, 0, 80.46973567850056, 0, -8.983152841195215e-05, 29.068135121181538]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [9129, 7501], 'crs': 'EPSG:4326', 'crs_transform': [8.983152841195215e-05, 0, 80.46973567850056, 0, -8.983152841195215e-05, 29.068135121181538]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [9129, 7501], 'crs': 'EPSG:4326', 'crs_transform': [8.983152841195215e-05, 0, 80.46973567850056, 0, -8.983152841195215e-05, 29.0681351211815

In [ ]:
# Function to rename the image bands based on the date
def rename_bands_by_date(image):
    # Get the acquisition date of the image
    date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')

    # Get the band names
    bands = image.bandNames()

    # Rename the bands using the date
    renamed_bands = bands.map(lambda band: ee.String(band).cat('_').cat(date))

    # Apply the renaming
    renamed_image = image.rename(renamed_bands)

    return renamed_image

# Apply the renaming function to the image collection
renamed_collection = image_collection.map(rename_bands_by_date)

# Get information on the renamed collection (optional: just to see the result)
info = renamed_collection.getInfo()
for img in info['features']:
    print(img['bands'])  # Print the renamed bands for each image

[{'id': 'B1_2024-02-02', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [9129, 7501], 'crs': 'EPSG:4326', 'crs_transform': [8.983152841195215e-05, 0, 80.46973567850056, 0, -8.983152841195215e-05, 29.068135121181538]}, {'id': 'B2_2024-02-02', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [9129, 7501], 'crs': 'EPSG:4326', 'crs_transform': [8.983152841195215e-05, 0, 80.46973567850056, 0, -8.983152841195215e-05, 29.068135121181538]}, {'id': 'B3_2024-02-02', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [9129, 7501], 'crs': 'EPSG:4326', 'crs_transform': [8.983152841195215e-05, 0, 80.46973567850056, 0, -8.983152841195215e-05, 29.068135121181538]}, {'id': 'B4_2024-02-02', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [9129, 7501], 'crs': 'EPSG:4326', 'crs_transform': [8.983152841195215e-05, 0, 80.46973567850056, 0, -8.983152841195215e-05, 29.068135121181538]}, {'id': 'B5_2024-02-02', 'data_t

There is one more appraoch


combining all the bands of the image collection into one image

In [ ]:
# kailali district boundary
palikas = ee.FeatureCollection("users/ssapkota913/Nepal_palika")
kailali_boundary = palikas.filter(
    ee.Filter.inList('DISTRICT', ['KAILALI'])
)


In [ ]:
combined_image = renamed_collection.toBands()
composite = combined_image.clip(kailali_boundary)
print(composite.getInfo())

In [ ]:
visParams = {'bands': ['0_B4_2024-02-02', '0_B3_2024-02-02',  '0_B2_2024-02-02'], 'min': 0, 'max': 0.3, 'gamma': 1.2}
Map.addLayer(composite, visParams, 'True Color Composite')
Map.addLayer(kailali_boundary)
Map.centerObject(kailali_boundary, 12)
Map

Map(center=[28.741496832416534, 80.88116230400202], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:

# gcp = ee.FeatureCollection("users/ssapkota913/kailali_poly_filtered")
gcp = ee.FeatureCollection("users/ssapkota913/cleaned_poly_kailali")
# composite = ee.Image('users/ssapkota913/sg_crop_winter_median_10m')
# Import and visualize the media composite and the kailali polygons
geometry = gcp.geometry()
palikas = ee.FeatureCollection("users/ssapkota913/Nepal_palika")
# kailali_palika = palikas.filter(
# ee.Filter.inList('PALIKA', ['Lamkichuha', 'Janaki'])
# )

# Import the crop mask from the asset
cropMask = ee.Image('users/ssapkota913/crop_mask_kailali_10m_winter').clip(kailali_boundary)

composite1 = composite.updateMask(cropMask)

# Get the distinct values of the 'crop_summe' column
uniqueCropSummer = gcp.aggregate_array('crop_summe').distinct()
print('Unique crop_summer values:', uniqueCropSummer.getInfo())

# Get the frequency of each distinct value in the 'crop_winte' column
cropSummerCounts = gcp.aggregate_array('crop_summe') \
.reduce(ee.Reducer.frequencyHistogram())

# Print the number of each unique crop_winte value
print('Number of each crop class in crop_summer: \n', cropSummerCounts.getInfo())

# Convert the frequency histogram to a dictionary
cropSummerDict = ee.Dictionary(cropSummerCounts)

# Extract the list of crop_winte values and their counts
cropSummerKeys = cropSummerDict.keys()
cropSummerValues = cropSummerDict.values()

# Create a list of valid crop_winte values (count >= 10)

def func_vue(key):
    count = ee.Number(cropSummerDict.get(key))
    return ee.Algorithms.If(count.gte(10), key, None)

validCropWinter = cropSummerKeys.map(func_vue

).filter(ee.Filter.neq('item', None));

# Filter the gcp collection to keep only 'crop_winte' values with counts >= 10
filteredGcpSummer = gcp.filter(ee.Filter.inList('crop_summe', validCropWinter))
print('Filtered GCP Summer : \n', filteredGcpSummer.getInfo())

# Get the frequency of each distinct value in the 'crop_winte' column
cropSummerCounts = filteredGcpSummer.aggregate_array('crop_summe') \
.reduce(ee.Reducer.frequencyHistogram())
# Print the number of each unique crop_winte value
print('Number of each crop class in crop_Summer:', cropSummerCounts.getInfo())

# Get the distinct values of the 'crop_winte' column
uniqueCropSummer = filteredGcpSummer.aggregate_array('crop_summe').distinct()
print('Unique filtered crop_summer values: \n', uniqueCropSummer.getInfo())

Map.addLayer(composite, visParams, 'True Color Composite')
Map.addLayer(composite1, visParams, 'TC_masked')
Map.addLayer(cropMask, {'min': 0, 'max': 1, 'palette': ['#7A87C6', '#E49635']}, 'Cultivable Land')
Map.addLayer(geometry, {'color': 'red'}, 'Croptype_label')
Map.centerObject(kailali_boundary, 12)

Unique crop_summer values: ['Dhaicha', 'banana', 'fallow', 'maize', 'shrub_tree', 'spices', 'vegetables, vegetables', 'black_gram', 'grassland', 'mustard', 'pulses', 'rice', 'vegetables', 'sugarcane', 'watermelon']
Number of each crop class in crop_summer: 
 {'Dhaicha': 1, 'banana': 11, 'black_gram': 2, 'fallow': 286, 'grassland': 12, 'maize': 71, 'mustard': 1, 'pulses': 1, 'rice': 12, 'shrub_tree': 11, 'spices': 1, 'sugarcane': 1, 'vegetables': 12, 'vegetables, vegetables': 8, 'watermelon': 1}
Filtered GCP Summer : 
 {'type': 'FeatureCollection', 'columns': {'_geolocati': 'String', '_meanmean': 'Float', 'crop_monso': 'String', 'crop_summe': 'String', 'crop_winte': 'String', 'district': 'String', 'harvest__1': 'String', 'harvest__2': 'String', 'harvest_da': 'String', 'id': 'Long', 'palika': 'String', 'remarks': 'String', 'shape': 'String', 'shape_area': 'Float', 'sowing_d_1': 'String', 'sowing_d_2': 'String', 'sowing_dat': 'String', 'system:index': 'String', 'yield_mons': 'Float', 'yie

In [ ]:
# Normalize the composite image (assuming you have a normalize function defined).
def normalize(image):
    bandNames = image.bandNames()
    minDict = image.reduceRegion(
    reducer = ee.Reducer.min(),
    geometry = kailali_boundary,
    scale = 10,
    maxPixels = 1e9,
    bestEffort = True,
    tileScale = 16
    )
    maxDict = image.reduceRegion(
    reducer = ee.Reducer.max(),
    geometry = kailali_boundary,
    scale = 10,
    maxPixels = 1e9,
    bestEffort = True,
    tileScale = 16
    )
    mins = ee.Image.constant(minDict.values(bandNames))
    maxs = ee.Image.constant(maxDict.values(bandNames))
    normalized = image.subtract(mins).divide(maxs.subtract(mins))
    return normalized

normalizedComposite = normalize(composite1)

print(normalizedComposite.getInfo())

# Remap the land cover class values to a 0-based sequential series.
classValues = ['fallow', 'vegetables', 'banana', 'grassland', 'maize', 'rice', 'shrub_tree']

remapValues = [0,1,2,3,4,5,6]

# Create a dictionary to map class values to remap values
classDict = ee.Dictionary.fromLists(classValues, remapValues)

print(classDict.getInfo())
# Function to remap the crop_winte values

def func_aju(feature):
    cropType = feature.get('crop_summe')
    remappedValue = classDict.get(cropType)
    return feature.set('crop_label', remappedValue)

remapFeature = func_aju

# Apply the remapping to each feature in the FeatureCollection
gcpRemappedSummer = filteredGcpSummer.map(remapFeature)

# Filter out any None values in the 'crop_label' field
validSamples = gcpRemappedSummer.filter(ee.Filter.notNull(['crop_label']))

# Overlay the point on the image to get training data.
sample = normalizedComposite.sampleRegions(
collection = validSamples,
properties = ['crop_label'],
scale = 10,
tileScale = 16
)

bands = normalizedComposite.bandNames()
print(bands.getInfo())

# Filter out samples with None values in any of the specified bands or properties.
filteredSample = sample.filter(ee.Filter.notNull(bands))

print('Size of the samples:', sample.size().getInfo())
print('Size of the filtered samples:', filteredSample.size().getInfo())

{'type': 'Image', 'bands': [{'id': '0_B1_2024-02-02', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [9129, 7501], 'crs': 'EPSG:4326', 'crs_transform': [8.983152841195215e-05, 0, 80.46973567850056, 0, -8.983152841195215e-05, 29.068135121181538]}, {'id': '0_B2_2024-02-02', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [9129, 7501], 'crs': 'EPSG:4326', 'crs_transform': [8.983152841195215e-05, 0, 80.46973567850056, 0, -8.983152841195215e-05, 29.068135121181538]}, {'id': '0_B3_2024-02-02', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [9129, 7501], 'crs': 'EPSG:4326', 'crs_transform': [8.983152841195215e-05, 0, 80.46973567850056, 0, -8.983152841195215e-05, 29.068135121181538]}, {'id': '0_B4_2024-02-02', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [9129, 7501], 'crs': 'EPSG:4326', 'crs_transform': [8.983152841195215e-05, 0, 80.46973567850056, 0, -8.983152841195215e-05, 29.06813512118

Here we will separate the training and testing dataset <br>
We are keeping the validation set pure and then perform balancing

In [ ]:
filteredSample = filteredSample.randomColumn()
trainingSample = filteredSample.filter('random <= 0.7')
validationSample = filteredSample.filter('random > 0.7')
print(validationSample.size().getInfo())

2773


Balancing the training dataset only dataset statstically mean, median ratio

In [ ]:
# Compute a histogram
hist = trainingSample.reduceColumns(ee.Reducer.frequencyHistogram(), ["crop_label"])
hist = ee.Dictionary(hist.get('histogram'))
values = hist.values()
print(hist.getInfo())

# Subset to balance number of features.
classes = hist.keys()
min = ee.Number(values.reduce(ee.Reducer.min()))
max = ee.Number(values.reduce(ee.Reducer.max()))
mean = ee.Number(values.reduce(ee.Reducer.mean()))
median = ee.Number(values.reduce(ee.Reducer.median()))

# Function to upsample features based on the majority class size
def func_upsample(c):
    # Calculate ratio based on the majority class size (max) or another class
    ratio = mean.divide(hist.getNumber(c))
    original = filteredSample.filter(ee.Filter.eq('crop_label', ee.Number.parse(c)))

    # Generate a list for replication based on the floor value of the ratio
    count = ratio.floor().toInt()
    reps = ee.List.sequence(0, count.subtract(1))

    # Use ee.List.map to replicate the original collection
    def replicateFeatures(i):
        return original.randomColumn()

    # Apply the replication and flatten the resulting collections
    upsampled = ee.FeatureCollection(reps.map(replicateFeatures)).flatten()

    # Add the remaining features proportionally up to the required ratio
    remaining = original.randomColumn().filter(ee.Filter.lt('random', ratio.mod(1)))

    return upsampled.merge(remaining)

# Apply the function to all classes and flatten the result to get a balanced dataset
upsampled_subset = classes.map(func_upsample)

# Flatten the list of upsampled collections
balanced_upsampled_training = ee.FeatureCollection(upsampled_subset).flatten()

# Print the new histogram for the upsampled dataset
print(balanced_upsampled_training.reduceColumns(ee.Reducer.frequencyHistogram(), ["crop_label"]).getInfo())

{'0': 4809, '1': 85, '2': 349, '3': 167, '4': 911, '5': 146, '6': 101}
{'histogram': {'0': 1360, '1': 1323, '2': 1369, '3': 1327, '4': 1335, '5': 1320, '6': 1295}}


In [ ]:
import statistics

# Class samples for each class
class_samples = [4809, 85,  349, 167, 911, 146,  101]

# Calculate the median from class_samples list
mean = statistics.mean(class_samples)
print("Mean:", mean)

# Compute the ratio for each class based on the median
ratios = [mean / sample for sample in class_samples]

# Print ratios for each class
for i, ratio in enumerate(ratios):
    print(f"Class {i+1}: Ratio = {ratio:.2f}")


Mean: 938.2857142857143
Class 1: Ratio = 0.20
Class 2: Ratio = 11.04
Class 3: Ratio = 2.69
Class 4: Ratio = 5.62
Class 5: Ratio = 1.03
Class 6: Ratio = 6.43
Class 7: Ratio = 9.29


In [ ]:
print(balanced_upsampled_training.size().getInfo())

9329


Using smote to balance the dataset
*   filteredSample - pandas dataframe
*   Balancing the data set using smote
*   pandas dataframe to feature collection



In [ ]:
# Add a random value field to the sample and use it to split 70% into training and 30% into validation.
# filteredSample = filteredSample.randomColumn()
# trainingSample = filteredSample.filter('random <= 0.6')
# validationSample = filteredSample.filter('random > 0.6')


# balanced_upsampled = balanced_upsampled.randomColumn()
# trainingSample = balanced_upsampled.filter('random <= 0.7')
# validationSample = balanced_upsampled.filter('random > 0.7')
trainingSample = balanced_upsampled_training
#band_names = ['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B11','B12','ndvi']
band_names = normalizedComposite.bandNames()

# Train a Random Forest classifier with 100 trees.
trainedClassifier = ee.Classifier.smileRandomForest(500).train(
features = trainingSample,
classProperty = 'crop_label',
inputProperties = band_names
)

print('First training samples results: \n', trainingSample.first().getInfo())
# print('Results of trained classifier \n', trainedClassifier.explain().getInfo())

# Get a confusion matrix and overall accuracy for the training sample.
trainAccuracy = trainedClassifier.confusionMatrix()
# print('Training error matrix', trainAccuracy.getInfo())
# print('Training overall accuracy', trainAccuracy.accuracy().getInfo())

# Get a confusion matrix and overall accuracy for the validation sample.
validationSample = validationSample.classify(trainedClassifier)
validationAccuracy = validationSample.errorMatrix('crop_label', 'classification')
# print('Validation error matrix', validationAccuracy.getInfo())
# print('Validation accuracy:', validationAccuracy.accuracy().getInfo())

# # Export the validation results to Google Drive
# Export.table.toDrive({
#     'collection': validation_results,
#     'description': 'validation_results',
#     'fileFormat': 'CSV'
# })

# Perform the validation and accuracy calculation outside Earth Engine



# Load the classifier after the export finishes and visualize.
#savedClassifier = ee.Classifier.load(assetId)

# Classify the reflectance image from the trained classifier.
imgClassified = normalizedComposite.classify(trainedClassifier)
masked_classified = imgClassified.updateMask(cropMask)
# print('Classified Image:', imgClassified.getInfo())

classVis = {
    'min': 0,
    'max': 6,
    'palette' : [
    'D3D3D3',
    'FFD700',
    '1E90FF',
    '8B4513',
    'FF69B4',
    '9370DB',
    '32CD32'
    ]
}

Map.centerObject(kailali_boundary, 12)
Map.addLayer(masked_classified, classVis, 'Classified')
# imgClassified = imgClassified.updateMask(cropMask)
# Define the export parameters
# Export.image.toDrive(
#   image = masked_classified.clip(kailali_palika),
#   description = 'Classified_Image',
#   scale = 10, # Adjust the scale based on your image resolution
#   region = kailali_palika, # Export the area of interest
#   fileNamePrefix = 'winter_ct_map',
#   folder = 'Crop_type_map', # Optional = specify the folder in your Google Drive
#   crs = 'EPSG =4326' # Coordinate Reference System
# })

# # Define the export parameters
# geemap.ee_export_image_to_asset(
# image = masked_classified.clip(kailali_palika),
# description = 'winter_crop_type',
# assetId = 'users/ssapkota913/winter_ct_map_kailali',
# scale = 10,
# region = kailali_palika,
# crs = 'EPSG =4326'
# )

# areaImage = ee.Image.pixelArea().addBands(masked_classified)

# # Calculate area per class
# areas = areaImage.reduceRegion(
# reducer = ee.Reducer.sum().group(
# groupField = 1,
# groupName = 'class',
# ),
# geometry = kailali_boundary.geometry(),
# scale = 10,
# maxPixels = 1e13
# )
# print(areas.getInfo())

# # Create an export classifier task to run.
# assetId = 'users/ssapkota913/winter_crop_classifier' ;  # <> modify these
# Export.classifier.toAsset(
#   classifier = trainedClassifier,
#   description = 'winter_crop_classifier',
#   assetId = assetId
# })

# Creates and styles 1 row of the legend.

Map

First training samples results: 
 {'type': 'Feature', 'geometry': None, 'id': '0_2_00000000000000000144_0', 'properties': {'0_B11_2024-02-02': 0.2739261798011414, '0_B11_fitted_2024-02-02': 0.29479958602323736, '0_B12_2024-02-02': 0.278659857580949, '0_B12_fitted_2024-02-02': 0.3045580059775934, '0_B1_2024-02-02': 0.2075785804605582, '0_B1_fitted_2024-02-02': 0.24228379690437063, '0_B2_2024-02-02': 0.22287335812393122, '0_B2_fitted_2024-02-02': 0.2066758454566609, '0_B3_2024-02-02': 0.22833098941293462, '0_B3_fitted_2024-02-02': 0.2194999496318501, '0_B4_2024-02-02': 0.16445751966255787, '0_B4_fitted_2024-02-02': 0.16640847059263925, '0_B5_2024-02-02': 0.217776274450226, '0_B5_fitted_2024-02-02': 0.2301562781771996, '0_B6_2024-02-02': 0.32596781830540283, '0_B6_fitted_2024-02-02': 0.33652579809749766, '0_B7_2024-02-02': 0.33658802830408685, '0_B7_fitted_2024-02-02': 0.3504982716399781, '0_B8A_2024-02-02': 0.3272419950384063, '0_B8A_fitted_2024-02-02': 0.33859351045675345, '0_B8_2024-02

Map(bottom=437123.0, center=[28.741496832416534, 80.88116230400202], controls=(WidgetControl(options=['positio…

In [ ]:
# # Calculate variable importance
# importance = ee.Dictionary(trainedClassifier.explain().get('importance'))

# # Calculate relative importance
# sum = importance.values().reduce(ee.Reducer.sum())

# def func_btt(key, val):
#     return (ee.Number(val).multiply(100)).divide(sum)

# relativeImportance = importance.map(func_btt)

# # Create a FeatureCollection from the relative importance dictionary
# importanceFc = ee.FeatureCollection([
#     ee.Feature(None, relativeImportance)
# ])

# # Export the FeatureCollection to Google Drive
# export_task = ee.batch.Export.table.toDrive(
#     collection = importanceFc,
#     description = 'Band_Importance_Export_Summer',
#     fileFormat = 'CSV'
# )

# # Start the export task
# export_task.start()
# print('Export started...')


# # List and check the status of all active tasks
# tasks = ee.batch.Task.list()
# for task in tasks:
#     print(task.status())

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# import pandas as pd
# feature_imp = pd.read_csv('/content/drive/MyDrive/Band_Importance_Export_Summer.csv')
# print(feature_imp.head())


Mounted at /content/drive


In [ ]:
# Get the confusion matrix
validationSample = validationSample.classify(trainedClassifier)
validationAccuracy = validationSample.errorMatrix('crop_label', 'classification')

# Extract the confusion matrix array
confusion_matrix_array = validationAccuracy.array()

print(confusion_matrix_array.getInfo())

[[1918, 4, 11, 7, 62, 1, 14], [0, 35, 0, 0, 0, 0, 0], [0, 0, 164, 0, 0, 0, 0], [0, 0, 0, 68, 0, 0, 0], [0, 0, 0, 0, 390, 0, 0], [0, 0, 0, 0, 0, 61, 0], [0, 0, 0, 0, 0, 0, 38]]


In [ ]:
# import geemap
# import pandas as pd

# # Assuming `validationSample` and `trainedClassifier` are already defined

# # Get the confusion matrix
# validationSample = validationSample.classify(trainedClassifier)
# validationAccuracy = validationSample.errorMatrix('crop_label', 'classification')

# # Extract the confusion matrix array
# confusion_matrix_array = validationAccuracy.array()

# print(confusion_matrix_array.getInfo())



exportAccuracy = ee.Feature(None, {'matrix': validationAccuracy.array()})

# Export the FeatureCollection.
geemap.ee_export_vector_to_drive(
collection = ee.FeatureCollection(exportAccuracy),
description = 'exportAccuracy_Summer_validation_sep',
fileFormat = 'CSV'
)


Exporting exportAccuracy_Summer_validation_sep... Please check the Task Manager from the JavaScript Code Editor.




```
# This is formatted as code
```

Export the classified model and image in assets

In [ ]:
# Define the asset ID for the classifier export
asset_id = 'users/ssapkota913/summer_kailali_crop_classifier_val_sep'

# Export the trained classifier to the specified asset
export_task = ee.batch.Export.classifier.toAsset(
    classifier=trainedClassifier,
    description='summer_kailali_crop_classifier_val_sep',
    assetId=asset_id
)

# Start the export task
export_task.start()

print('Export task started.')

Export task started.


In [ ]:
# Define the export parameters
geemap.ee_export_image_to_asset(
image = masked_classified.clip(kailali_boundary.geometry()),
description = 'summer_crop_type',
assetId = 'users/ssapkota913/summer_ct_kailali_dist_val_sep',
scale = 10,
region = kailali_boundary.geometry(),
crs = 'EPSG:4326'
)


In [ ]:
image1 = ee.Image('users/ssapkota913/summer_ct_kailali_dist')
classVis = {
    'min': 0,
    'max': 6,
    'palette' : [
    'D3D3D3',
    'FFD700',
    '1E90FF',
    '8B4513',
    'FF69B4',
    '9370DB',
    '32CD32'
    ]
}

legend_keys = ['Fallow', 'Vegetable', 'Banana', 'Grassland', 'Maize','Rice', 'Shrub tree']
# colorS can be defined using either hex code or RGB (0-255, 0-255, 0-255)
legend_colors = [
    'D3D3D3',
    'FFD700',
    '1E90FF',
    '8B4513',
    'FF69B4',
    '9370DB',
    '32CD32'
    ]

Map.add_legend(keys=legend_keys, colors=legend_colors, position="bottomleft")
# Add the image layer to the map
Map.addLayer(image1, classVis, name='Summer Crop Type Map')

In [ ]:
geemap.ee_export_image_to_drive(
    image = image1.unmask(-9999.0),
    description = 'summer_kailali',
    folder = 'Crop_type_map',  # Change this to the desired folder name in your Google Drive
    fileNamePrefix = 'summer_ct_kailali_dist',  # Prefix for the exported file name
    scale = 10,
    region = kailali_boundary.geometry(),
    crs = 'EPSG:4326'
)

In [ ]:
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
areaImage = ee.Image.pixelArea().addBands(masked_classified)

# Calculate area per class
areas = areaImage.reduceRegion(
reducer = ee.Reducer.sum().group(
groupField = 1,
groupName = 'class',
),
geometry = kailali_boundary.geometry(),
scale = 10,
maxPixels = 1e13
)
print(areas.getInfo())

{'groups': [{'class': 0, 'sum': 613066760.2444056}, {'class': 1, 'sum': 3731407.31976806}, {'class': 2, 'sum': 54061974.576575905}, {'class': 3, 'sum': 8563908.359544164}, {'class': 4, 'sum': 102570170.17804597}, {'class': 5, 'sum': 3963107.7617819393}, {'class': 6, 'sum': 10940042.746721365}]}


In [ ]:
# separating the validation and then balancing.

import numpy as np

# Confusion matrix
conf_matrix = np.array(
[[1918, 4, 11, 7, 62, 1, 14],
[0, 35, 0, 0, 0, 0, 0],
[0, 0, 164, 0, 0, 0, 0],
[0, 0, 0, 68, 0, 0, 0],
[0, 0, 0, 0, 390, 0, 0],
[0, 0, 0, 0, 0, 61, 0],
[0, 0, 0, 0, 0, 0, 38]]
)

# Calculate precision, recall, and F1 score for each class
precision = []
recall = []
f1_score = []

# Number of classes
n_classes = conf_matrix.shape[0]

for i in range(n_classes):
    # True positives are the diagonal elements
    TP = conf_matrix[i, i]

    # False positives are the sum of the column (minus the true positives)
    FP = conf_matrix[:, i].sum() - TP

    # False negatives are the sum of the row (minus the true positives)
    FN = conf_matrix[i, :].sum() - TP

    # Precision
    precision_i = TP / (TP + FP) if (TP + FP) != 0 else 0
    precision.append(precision_i)

    # Recall
    recall_i = TP / (TP + FN) if (TP + FN) != 0 else 0
    recall.append(recall_i)

    # F1 Score
    if precision_i + recall_i != 0:
        f1_i = 2 * (precision_i * recall_i) / (precision_i + recall_i)
    else:
        f1_i = 0
    f1_score.append(f1_i)

# Print the results
for i in range(n_classes):
    print(f"Class {i}:")
    print(f"  Precision: {precision[i]:.4f}")
    print(f"  Recall: {recall[i]:.4f}")
    print(f"  F1 Score: {f1_score[i]:.4f}\n")

# Calculate overall accuracy
true_positives = np.trace(conf_matrix)  # Sum of diagonal elements
total_predictions = conf_matrix.sum()  # Sum of all elements
accuracy = true_positives / total_predictions

# Print overall accuracy
print(f"Overall Accuracy: {accuracy:.4f}")


Class 0:
  Precision: 1.0000
  Recall: 0.9509
  F1 Score: 0.9748

Class 1:
  Precision: 0.8974
  Recall: 1.0000
  F1 Score: 0.9459

Class 2:
  Precision: 0.9371
  Recall: 1.0000
  F1 Score: 0.9676

Class 3:
  Precision: 0.9067
  Recall: 1.0000
  F1 Score: 0.9510

Class 4:
  Precision: 0.8628
  Recall: 1.0000
  F1 Score: 0.9264

Class 5:
  Precision: 0.9839
  Recall: 1.0000
  F1 Score: 0.9919

Class 6:
  Precision: 0.7308
  Recall: 1.0000
  F1 Score: 0.8444

Overall Accuracy: 0.9643
